<a href="https://colab.research.google.com/github/earandir/DataAnalysis/blob/main/Modulo03/M03S07R01casting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejemplo 1: Casting

### 1. Objetivos:
    - Aprender a usar `astype`
    - Aprender a lidiar con errores usando `to_numeric`
    - Aprender a convertir `strings` e `ints` a `datetime`
 
---
    
### 2. Desarrollo:

In [29]:
import pandas as pd
import numpy as np


Vamos a trabajar con el conjunto de datos `new_york_times_bestsellers-dirty.csv` que está en la carpeta `DataFrames`, sería conveniente revisar el contenido del archivo antes de importarlo como un `DataFrame` y observa como existe una columna que puede ser usada como índice, así que ahora leeremos el archivo usando la función `read_csv()` de la siguiente forma:

`pd.read_csv(-nombre de archivo-, index_col=-número de columna que se usará como índice-)`

en este caso es la columna `0`:

In [2]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

data = '/content/drive/MyDrive/DataAnalysis/Datasets/new_york_times_bestsellers-dirty.csv'

Mounted at /content/drive


In [3]:
df = pd.read_csv(data, index_col=0)

In [4]:
df

df.head()

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Descr: Aliens have taken control of the minds ...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,2008-05-24 00:00:00,1212883200000,2,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,Descr: A woman's happy marriage is shaken when...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,2008-05-24 00:00:00,1212883200000,3,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,Descr: A Massachusetts state investigator and ...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,2008-05-24 00:00:00,1212883200000,4,0,1,22.95
3,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,Descr: An aging porn queens aims to cap her ca...,Doubleday,SNUFF,5b4aa4ead3089013507db18f,2008-05-24 00:00:00,1212883200000,5,0,1,24.95
5,http://www.amazon.com/Phantom-Prey-John-Sandfo...,John Sandford,Descr: The Minneapolis detective Lucas Davenpo...,Putnam,PHANTOM PREY,5b4aa4ead3089013507db191,2008-05-24 00:00:00,1212883200000,7,4,3,26.95


Tenemos aquí un dataset donde no todos los tipos de datos han sido deducidos correctamente, así que primero vamos a obtener los tipos de datos asignados por Pandas para cada conlumna con:

`dataframe.dtypes`

In [5]:
df.dtypes

amazon_product_url              object
author                          object
description                     object
publisher                       object
title                           object
oid                             object
bestsellers_date.numberLong     object
published_date.numberLong        int64
rank.numberInt                  object
rank_last_week.numberInt         int64
weeks_on_list.numberInt          int64
price.numberDouble             float64
dtype: object

Específicamente, tenemos dos columnas con fechas (`bestsellers_date.numberLong` y `published_date.numberLong`)  que tienen tipos `object` e `int64`. También tenemos una columna `rank.numberInt` que no tiene el tipo de dato adecuado.

Podemos usar el método `astype` para pasarle a nuestro `DataFrame` un `diccionario` de conversión. Por ejemplo, vamos a convertir nuestras dos columnas de fechas usando un `diccionario` de conversión. El tipo de dato que usamos para manejar fechas es el llamado `datetime`. Este tipo de dato nos permite manipular fechas y horarios muy eficientemente.

In [6]:
diccionario_de_conversion = {
    "bestsellers_date.numberLong":"datetime64[ns]",
    "published_date.numberLong":"datetime64[ns]",
}

Bueno, hagamos una disertación sobre como funcionan las fechas en Python, primero para usar fechas hay que hacer uso del módulo `datetime` (Pandas lo usa en automático), luego a partir de este módulo existen los tipos de datos `datetime`, `date` y `time` para manejar fechas-hora, fechas y horas en Python.

Por ejemplo, vamos a crear tres variables, con la fecha-hora, lafecha y la hora actual:

In [7]:
import datetime

timestamp = datetime.datetime.now()
hoy = timestamp.now()
fecha = timestamp.date()
hora = timestamp.time()

print(hoy, type(hoy))
print(fecha, type(fecha))
print(hora, type(hora))

2021-07-14 16:42:43.686454 <class 'datetime.datetime'>
2021-07-14 <class 'datetime.date'>
16:42:43.686409 <class 'datetime.time'>


Y también podemos acceder a los atributos de cada uno de estos tipos de datos como son **año**, **mes**, **día**, **hora**, **minuto**, **segundo** entre otros, vamos a imprimir lo siguiente:

- Año y hora usando la variable `hoy`
- La fecha en formato México usando la variable `fecha` y una `f-string`
- La hora con horas, minutos y segundos usando la variable `hora` con la función `hora.strftime(-formato-)` 

In [8]:
print(hoy.year, hoy.hour)
print(f'{hoy.day}/{hoy.month}/{hoy.year}')
print(hora.strftime("%H:%M:%S"))

2021 16
14/7/2021
16:42:43


Entonces, regresando a nuestro dataframe, aplicamos el diccionario de conversión usando la forma:

`dataframe.astype(-diccionario-)`

In [9]:
df_1 = df.astype(diccionario_de_conversion)

df_1.head()

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Descr: Aliens have taken control of the minds ...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,2008-05-24,1970-01-01 00:20:12.883200,2,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,Descr: A woman's happy marriage is shaken when...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,2008-05-24,1970-01-01 00:20:12.883200,3,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,Descr: A Massachusetts state investigator and ...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,2008-05-24,1970-01-01 00:20:12.883200,4,0,1,22.95
3,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,Descr: An aging porn queens aims to cap her ca...,Doubleday,SNUFF,5b4aa4ead3089013507db18f,2008-05-24,1970-01-01 00:20:12.883200,5,0,1,24.95
5,http://www.amazon.com/Phantom-Prey-John-Sandfo...,John Sandford,Descr: The Minneapolis detective Lucas Davenpo...,Putnam,PHANTOM PREY,5b4aa4ead3089013507db191,2008-05-24,1970-01-01 00:20:12.883200,7,4,3,26.95


Se examinan nuevamente los tipos de las columnas con `df.dtypes`:

In [10]:
df_1.dtypes

amazon_product_url                     object
author                                 object
description                            object
publisher                              object
title                                  object
oid                                    object
bestsellers_date.numberLong    datetime64[ns]
published_date.numberLong      datetime64[ns]
rank.numberInt                         object
rank_last_week.numberInt                int64
weeks_on_list.numberInt                 int64
price.numberDouble                    float64
dtype: object

Como puedes ver, nuestras columnas han sido transformadas. Pero parece que hay un problema, puesto que hay muchísima diferencia de años entre la columna `bestsellers_date` y la columna `published_date`. Esto se debe a que `published_date` está en formato 'milisegundos desde La Época (la medianoche UTC del 1 de enero de 1970)' y `pandas` asume por default que estamos lidiando con nanosegundos.

Vamos a resolverlo de dos formas, la matemática (no entrar en pánico) y la forma **pandesca**, para la primera hay que saber que en unidades de medida existe los segundos, milisegundos (o la milesima parte), los microsegundos (o la millonésima parte) y los nanosegundos (o la mil millonésima parte), otra forma de verlo es que la siguiente:

- 1 seg = 1 x 1000 milisegundos
- 1 seg = 1 x 1000 000 microsegundos
- 1 seg = 1 x 1000 000 000 nanosegundos

así que si queremos pasar de milisegundos a nanosegundos hay que multiplicar por un millón (1000 000) la columna de `published_date` y luego se aplica la función `df.astype()`:

In [11]:
df_2 = df.copy()
df_2["published_date.numberLong"] = pd.to_datetime(df["published_date.numberLong"], unit='ms')

df_2.astype(diccionario_de_conversion).head(3)

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Descr: Aliens have taken control of the minds ...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,2008-05-24,2008-06-08,2,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,Descr: A woman's happy marriage is shaken when...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,2008-05-24,2008-06-08,3,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,Descr: A Massachusetts state investigator and ...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,2008-05-24,2008-06-08,4,0,1,22.95


Genial las fechas ya están mucho mejor, ahora la forma usando **Pandas** es por medio de la función `pd.to_datetime` en la forma:

`pd.to_datatime(-serie-, unit="ns")`

- `-serie-` es la columna del dataframe a convertir
- `unit="ns"` indicamos las unidades de nuestros datos, en este caso usaremos ms = milisegundos

In [12]:
pd.to_datetime(df["published_date.numberLong"], unit='ms')

0      2008-06-08
1      2008-06-08
2      2008-06-08
3      2008-06-08
5      2008-06-08
          ...    
3027   2013-05-05
3028   2013-05-05
3029   2013-05-05
3030   2013-05-05
3031   2013-05-05
Name: published_date.numberLong, Length: 2266, dtype: datetime64[ns]

Ahora remplacemos los valores de la columna `published_date` con las nuevas fechas: 

In [13]:
df_1["published_date.numberLong"] = pd.to_datetime(df["published_date.numberLong"], unit='ms')

df_1.head(3)

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Descr: Aliens have taken control of the minds ...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,2008-05-24,2008-06-08,2,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,Descr: A woman's happy marriage is shaken when...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,2008-05-24,2008-06-08,3,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,Descr: A Massachusetts state investigator and ...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,2008-05-24,2008-06-08,4,0,1,22.95


Muy bien, las fechas ya son manejables por Python ahora, vamos a corroborar los tipos en el dataframe usando `df.dtypes`:

In [14]:
df_1.dtypes

amazon_product_url                     object
author                                 object
description                            object
publisher                              object
title                                  object
oid                                    object
bestsellers_date.numberLong    datetime64[ns]
published_date.numberLong      datetime64[ns]
rank.numberInt                         object
rank_last_week.numberInt                int64
weeks_on_list.numberInt                 int64
price.numberDouble                    float64
dtype: object

La columna `rank.numberInt` aparece como **object**, pero contiene datos de tipo entero, así que vamos a realizar la conversión usando `astype` en la forma:

`dataframe[-columna-].astype(-nuevo tipo de dato-)`

In [15]:
df_1['rank.numberInt'].astype('int64')

ValueError: ignored

No podemos hacerlo porque hay unos valores tipo `string` que no pueden ser convertidos a `int`. Para esto usamos el método `to_numeric`, que nos permite indicar que cuando un error sea encontrado, debe de ser sustituido por un `NaN` en la forma:

`pd.to_numeric(-columna-, errors="coerce")`

In [16]:
pd.to_numeric(df["rank.numberInt"], errors='coerce')

0        2.0
1        3.0
2        4.0
3        5.0
5        7.0
        ... 
3027     8.0
3028     9.0
3029    11.0
3030    13.0
3031    14.0
Name: rank.numberInt, Length: 2266, dtype: float64

Vamos a reasignar el resultado al `DataFrame` original:

In [17]:
df_1['rank.numberInt'] = pd.to_numeric(df["rank.numberInt"], errors='coerce')

df_1.head(3)

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Descr: Aliens have taken control of the minds ...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,2008-05-24,2008-06-08,2.0,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,Descr: A woman's happy marriage is shaken when...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,2008-05-24,2008-06-08,3.0,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,Descr: A Massachusetts state investigator and ...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,2008-05-24,2008-06-08,4.0,0,1,22.95


Ahora, para convertilo a tipo `int` podemos eliminar los `NaNs` con `df.dropna(axis=0)` y luego usar `-columna-.astype(int)`:

In [18]:
df_1 = df_1.dropna(axis=0)
df_1["rank.numberInt"] = df_1['rank.numberInt'].astype('int64')

df_1.head(3)

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Descr: Aliens have taken control of the minds ...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,2008-05-24,2008-06-08,2,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,Descr: A woman's happy marriage is shaken when...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,2008-05-24,2008-06-08,3,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,Descr: A Massachusetts state investigator and ...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,2008-05-24,2008-06-08,4,0,1,22.95


Y se examina nuevamente los tipos de cada columna con `df.dtypes`:

In [19]:
df_1.dtypes

amazon_product_url                     object
author                                 object
description                            object
publisher                              object
title                                  object
oid                                    object
bestsellers_date.numberLong    datetime64[ns]
published_date.numberLong      datetime64[ns]
rank.numberInt                          int64
rank_last_week.numberInt                int64
weeks_on_list.numberInt                 int64
price.numberDouble                    float64
dtype: object

¡Listo!

---
---

## Reto 1: Casting

### 1. Objetivos:
    - Aplicar diversas técnicas de casting a un dataset nuevo
 
---
    
### 2. Desarrollo:

#### a) Transformando tipos de datos

Vamos a trabajar con una versión un poco modificada del dataset que creaste en la sesión pasada. Si bien recuerdas, al final de la sesión pasada automatizamos un programa de Python para obtener un `DataFrame` con todos los objetos que orbitaron cerca de la Tierra en Enero y Febrero de 1995. Para construir este dataset, usamos el API gratuito que ofrece la [NASA](https://api.nasa.gov/).

Me tomé la libertad de modificar un poco dicho dataset para que pudiera ser utilizado más efectivamente para los fines de esta sesión. Encontrarás la versión modificada en la ruta '../../Datasets/near_earth_objects-jan_feb_1995-dirty.csv'. Todos los Retos de esta sesión los harás con ese conjunto de datos.

Te recomiendo que al finalizar cada reto guardes la nueva versión modificada de tu dataset bajo un nombre que indique el reto realizado (por ejemplo, 'near_earth_objects-jan_feb_1995-reto_1.csv'), para que puedas ir trabajando incrementalmente a través de los retos y no tengas que repetir procesos. Puedes guardar conjuntos de datos en formato `csv` usando el método `DataFrame.to_csv('ruta')`.

Tu primer Reto consistirá en seguir los siguientes pasos:

1. Lee el dataset y crea un `DataFrame` con él.

In [24]:
data = '/content/drive/MyDrive/DataAnalysis/Datasets/near_earth_objects-jan_feb_1995-dirty.csv'
df_reto_1 = pd.read_csv(data, index_col=0)
df_reto_1.head(3)

,id_name,is_potentially_hazardous_asteroid,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,orbit_class_description
0,2154652-154652 (2004 EP20),False,483.676488,1081.533507,1995-01-07,789467580000,earth,16.142864,58114.3086669449,Near-Earth-asteroid-orbits-similar-to-that-o...
1,3153509-(2003 HM),True,96.506147,215.794305,1995-01-07,789491340000,earth,12.351044,44463.7577343496,Near-Earth-asteroid-orbits-which-cross-the-E...
2,3516633-(2010 HA),False,44.111820,98.637028,1995-01-07,789446820000,earth,6.220435,Unknown,Near-Earth-asteroid-orbits-similar-to-that-o...


2. Realiza una pequeña exploración para familiarizarte con él.

In [25]:
df_reto_1.dtypes

id_name                                              object
is_potentially_hazardous_asteroid                      bool
estimated_diameter.meters.estimated_diameter_min    float64
estimated_diameter.meters.estimated_diameter_max    float64
close_approach_date                                  object
epoch_date_close_approach                             int64
orbiting_body                                        object
relative_velocity.kilometers_per_second             float64
relative_velocity.kilometers_per_hour                object
orbit_class_description                              object
dtype: object

3. Convierte la columna `relative_velocity.kilometers_per_hour` de `object` a `float64` y elimina `NaNs`:

In [35]:
df_reto_1['relative_velocity.kilometers_per_hour'].replace('Unknown', np.nan, inplace=True)

In [49]:
df_reto_1.dropna(inplace=True)

In [44]:
df_reto_1['relative_velocity.kilometers_per_hour'] = df_reto_1['relative_velocity.kilometers_per_hour'].astype('float64')

In [51]:
df_reto_1.dtypes

id_name                                              object
is_potentially_hazardous_asteroid                      bool
estimated_diameter.meters.estimated_diameter_min    float64
estimated_diameter.meters.estimated_diameter_max    float64
close_approach_date                                  object
epoch_date_close_approach                             int64
orbiting_body                                        object
relative_velocity.kilometers_per_second             float64
relative_velocity.kilometers_per_hour               float64
orbit_class_description                              object
dtype: object

4. Convierte la columna `close_approach_date` a tipo de dato `datetime64[ms]` usando el método `astype` y un diccionario de conversión.

In [56]:
conver_dict = {'close_approach_date':'datetime64[ns]'}
df_reto_1 = df_reto_1.astype(conver_dict)     

df_reto_1.dtypes

id_name                                                     object
is_potentially_hazardous_asteroid                             bool
estimated_diameter.meters.estimated_diameter_min           float64
estimated_diameter.meters.estimated_diameter_max           float64
close_approach_date                                 datetime64[ns]
epoch_date_close_approach                                    int64
orbiting_body                                               object
relative_velocity.kilometers_per_second                    float64
relative_velocity.kilometers_per_hour                      float64
orbit_class_description                                     object
dtype: object

5. Convierte la columna `epoch_date_close_approach` a tipo de dato `datetime64[ms]` usando el método `to_datetime`:

In [57]:
df_reto_1['epoch_date_close_approach'] = pd.to_datetime(df_reto_1['epoch_date_close_approach'], unit='ms')
df_reto_1.dtypes

id_name                                                     object
is_potentially_hazardous_asteroid                             bool
estimated_diameter.meters.estimated_diameter_min           float64
estimated_diameter.meters.estimated_diameter_max           float64
close_approach_date                                 datetime64[ns]
epoch_date_close_approach                           datetime64[ns]
orbiting_body                                               object
relative_velocity.kilometers_per_second                    float64
relative_velocity.kilometers_per_hour                      float64
orbit_class_description                                     object
dtype: object

6. Reinicia los índices con `df.reset_index(drop=True)`:

In [58]:
df_reto_1.reset_index(drop=True)

df_reto_1.head()

,id_name,is_potentially_hazardous_asteroid,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,orbit_class_description
0,2154652-154652 (2004 EP20),False,483.676488,1081.533507,1995-01-07,1995-01-07 08:33:00,earth,16.142864,58114.308667,Near-Earth-asteroid-orbits-similar-to-that-o...
1,3153509-(2003 HM),True,96.506147,215.794305,1995-01-07,1995-01-07 15:09:00,earth,12.351044,44463.757734,Near-Earth-asteroid-orbits-which-cross-the-E...
3,3837644-(2019 AY3),False,46.190746,103.285648,1995-01-07,1995-01-07 21:25:00,earth,22.478615,80923.015021,Near-Earth-asteroid-orbits-similar-to-that-o...
4,3843493-(2019 PY),False,22.108281,49.435619,1995-01-07,1995-01-07 02:45:00,earth,4.998691,17995.288355,Near-Earth-asteroid-orbits-similar-to-that-of...
5,3765015-(2016 WR48),False,160.160338,358.129403,1995-01-08,1995-01-08 12:46:00,earth,7.465089,26874.321682,An-asteroid-orbit-contained-entirely-within-t...


7. Guarda tu resultado en un archivo .csv.

In [61]:
path = '/content/drive/MyDrive/DataAnalysis/new_york_times_bestsellers-clean.csv'
df_reto_1.to_csv(path)

En este caso, la celda de validación sólo verifica que los tipos de datos de las columnas de tu DataFrame sean las correctas, si tienes alguna duda de tu resultado pregunta a quién más confianza le tengas!!

In [59]:
def checar_conversiones(df):
    
    import pandas as pd
    import pandas.api.types as ptypes
    
    assert ptypes.is_float_dtype(df['relative_velocity.kilometers_per_hour']), 'Cuidado... La columna `relative_velocity.kilometers_per_hour` no es de tipo `float64`'
    assert ptypes.is_datetime64_any_dtype(df['close_approach_date']), 'Cuidado... La columna `close_approach_date` no es de tipo `datetime64[ns]`'
    assert ptypes.is_datetime64_any_dtype(df['epoch_date_close_approach']), 'Cuidado... La columna `epoch_date_close_approach` no es de tipo `datetime64[ns]'
    
    print(f'¡Éxito! ¡Todas tus conversiones fueron realizadas adecuadamente!')

checar_conversiones(df_reto_1)

¡Éxito! ¡Todas tus conversiones fueron realizadas adecuadamente!
